In [63]:

# image files. There are 43 unique classes in total. The images are of shape
# (30,30,3).
# ==============================================================================

# HINT: Your neural network must have a validation accuracy of approximately
# 0.95 or above on the normalized validation dataset for top marks.
 

In [64]:
import urllib
import zipfile
 
import tensorflow as tf
 
# This function downloads and extracts the dataset to the directory that
# contains this file.
# DO NOT CHANGE THIS CODE
# (unless you need to change https to http)


In [65]:
import wget

def download_and_extract_data():
   url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/germantrafficsigns.zip'
   wget.download(url)
   #urllib.request.urlretrieve(url, 'germantrafficsigns.zip')
   with zipfile.ZipFile('germantrafficsigns.zip', 'r') as zip_ref:
       zip_ref.extractall()
down_file = download_and_extract_data() 

In [66]:
 # COMPLETE THE CODE IN THIS FUNCTION
def preprocess(image, label):
   # NORMALIZE YOUR IMAGES HERE (HINT: Rescale by 1/.255)
   image = image /255
   labelZip = label
   return image, labelZip
 
 
# This function loads the data, normalizes and resizes the images, splits it into
# train and validation sets, defines the model, compiles it and finally
# trains the model. The trained model is returned from this function.
import os
base_dir = "./"
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

train_fname = os.listdir(train_dir)
val_fname = os.listdir(val_dir)
print(len(train_fname))
print(len(val_fname))
print(train_fname[:10])



43
43
['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009']


In [67]:

 
# COMPLETE THE CODE IN THIS FUNCTION.
def solution_model(train_dir, val_dir):
   # Downloads and extracts the dataset to the directory that
   # contains this file.
      
   # download_and_extract_data()
 
   BATCH_SIZE = 32
   IMG_SIZE = 30
 
   # The following code reads the training and validation data from their
   # respective directories, resizes them into the specified image size
   # and splits them into batches. You must fill in the image_size
   # argument for both training and validation data.
   # HINT: Image size is a tuple
   train_ds = tf.keras.preprocessing.image_dataset_from_directory(
       directory=train_dir,
       label_mode='categorical',
       image_size=  (30, 30),
       batch_size = BATCH_SIZE)
 
   val_ds = tf.keras.preprocessing.image_dataset_from_directory(
       directory=val_dir,
       label_mode='categorical',
       image_size= (30, 30),
       batch_size = BATCH_SIZE,)
   
   return train_ds, val_ds
 


In [68]:
train_ds, val_ds = solution_model(train_dir, val_dir)

Found 31367 files belonging to 43 classes.
Found 7842 files belonging to 43 classes.


In [69]:

   # Normalizes train and validation datasets using the
   # preprocess() function.
   # Also makes other calls, as evident from the code, to prepare them for
   # training.
   # Do not batch or resize the images in the dataset here since it's already
   # been done previously.
def img_process(train_ds, val_ds):
        train_img = train_ds.map(
            preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
            tf.data.experimental.AUTOTUNE)
        val_img = val_ds.map(
            preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        
        return train_img, val_img

train_img, val_img = img_process(train_ds, val_ds)
 

In [70]:
# Code to define the model
model = tf.keras.models.Sequential([
    # ADD LAYERS OF THE MODEL HERE

    # If you don't adhere to the instructions in the following comments,
    # tests will fail to grade your model:
    # The input layer of your model must have an input shape of
    # (30,30,3).
    # Make sure your last layer has 43 neurons activated by softmax.
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(30, 30, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(43, activation='softmax')

])

print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 12544)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               6423040   
                                                                 
 dense_5 (Dense)             (None, 43)                22059     
                                                                 
Total params: 6,446,891
Trainable params: 6,446,891
Non-trainable params: 0
____________________________________________

In [71]:
# Code to compile and train the model
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


model.fit(train_img, epochs = 10)
    # YOUR CODE HERE


 

Epoch 1/10
981/981 [==============================] - 37s 37ms/step - loss: 0.7727 - accuracy: 0.7968
Epoch 2/10
981/981 [==============================] - 33s 34ms/step - loss: 0.1725 - accuracy: 0.9552
Epoch 3/10
981/981 [==============================] - 33s 34ms/step - loss: 0.0989 - accuracy: 0.9740
Epoch 4/10
981/981 [==============================] - 33s 33ms/step - loss: 0.0783 - accuracy: 0.9794
Epoch 5/10
981/981 [==============================] - 33s 34ms/step - loss: 0.0608 - accuracy: 0.9834
Epoch 6/10
981/981 [==============================] - 33s 34ms/step - loss: 0.0437 - accuracy: 0.9887
Epoch 7/10
981/981 [==============================] - 33s 34ms/step - loss: 0.0407 - accuracy: 0.9899
Epoch 8/10
981/981 [==============================] - 33s 34ms/step - loss: 0.0383 - accuracy: 0.9907
Epoch 9/10
981/981 [==============================] - 33s 33ms/step - loss: 0.0422 - accuracy: 0.9898
Epoch 10/10
981/981 [==============================] - 33s 34ms/step - loss: 0.023

In [ ]:
# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
'''if __name__ == '__main__':
   model = solution_model()
   model.save("mymodel.h5")
'''

In [73]:
model.save("mymodelq3.h5")